In [6]:
import mysql.connector as SQLC
def search_db(query):
    cnx = SQLC.connect(
            host ="geo.di.uminho.pt",
            user ="bioinformatica",
            password ="20221207",
            database ="AP_db_KRG")
    cursor = cnx.cursor()
    cursor.execute("SELECT sequence FROM Gene")
    sequences = cursor.fetchall()
    cursor.close()
    return sequences
sequences=search_db("A")

In [3]:
import mysql.connector as SQLC

class Blast:
    def __init__(self, window):
        self.window = window
        self.cnx = SQLC.connect(
            host ="geo.di.uminho.pt",
            user ="bioinformatica",
            password ="20221207",
            database ="AP_db_KRG"
        )
    def search_db(self, query):
        cursor = self.cnx.cursor()
        cursor.execute("SELECT sequence FROM Gene WHERE sequence LIKE %s", ('%'+query+'%',))
        sequences = cursor.fetchall()
        cursor.close()
        if sequences:
            self.sequence = sequences[0][0]
            return True
        else:
            return False

    def hits(self, query_map, sequence):
        hits = []
        for i in range(len(sequence) - self.window):
            window = sequence[i:i+self.window]
            score = 0
            for q in query_map:
                if window.count(q) > 0:
                    score += 1
            if score == len(query_map):
                hits.append((i, window))
        return hits

    def query_map(self, query):
        return set(query)

    def run_blast(self, query):
        if self.search_db(query):
            qm = self.query_map(query)
            all_hits = self.hits(qm, self.sequence)
            return all_hits
        else:
            return "No results found."




In [4]:
# Usage example
blast = Blast(20)
query = input("Enter a sequence: ")
results = blast.run_blast(query)
print(results)

Enter a sequence: AAA
[(0, 'TACCTTGTCCGAAAAGGAAA'), (1, 'ACCTTGTCCGAAAAGGAAAG'), (2, 'CCTTGTCCGAAAAGGAAAGA'), (3, 'CTTGTCCGAAAAGGAAAGAC'), (4, 'TTGTCCGAAAAGGAAAGACA'), (5, 'TGTCCGAAAAGGAAAGACAA'), (6, 'GTCCGAAAAGGAAAGACAAA'), (7, 'TCCGAAAAGGAAAGACAAAT'), (8, 'CCGAAAAGGAAAGACAAATC'), (9, 'CGAAAAGGAAAGACAAATCA'), (10, 'GAAAAGGAAAGACAAATCAA'), (11, 'AAAAGGAAAGACAAATCAAG'), (12, 'AAAGGAAAGACAAATCAAGA'), (13, 'AAGGAAAGACAAATCAAGAA'), (14, 'AGGAAAGACAAATCAAGAAG'), (15, 'GGAAAGACAAATCAAGAAGC'), (16, 'GAAAGACAAATCAAGAAGCA'), (17, 'AAAGACAAATCAAGAAGCAA'), (18, 'AAGACAAATCAAGAAGCAAA'), (19, 'AGACAAATCAAGAAGCAAAC'), (20, 'GACAAATCAAGAAGCAAACT'), (21, 'ACAAATCAAGAAGCAAACTG'), (22, 'CAAATCAAGAAGCAAACTGC'), (23, 'AAATCAAGAAGCAAACTGCT'), (24, 'AATCAAGAAGCAAACTGCTT'), (25, 'ATCAAGAAGCAAACTGCTTT'), (26, 'TCAAGAAGCAAACTGCTTTG'), (27, 'CAAGAAGCAAACTGCTTTGG'), (28, 'AAGAAGCAAACTGCTTTGGT'), (29, 'AGAAGCAAACTGCTTTGGTT')]


In [8]:
class Blast:
    def __init__(self, w):
        self.w = w
        self.cnx = SQLC.connect(
            host ="geo.di.uminho.pt",
            user ="bioinformatica",
            password ="20221207",
            database ="AP_db_KRG"
        )

    def search_db(self, query):
        cursor = self.cnx.cursor()
        cursor.execute("SELECT sequence FROM Gene WHERE sequence LIKE %s", ('%'+query+'%',))
        sequences = cursor.fetchall()
        cursor.close()
        if sequences:
            self.sequences = sequences
            return True
        else:
            return False

    def query_map(self, query):
        tam = len(query)
        res = {}
        for chave, offset in [(query[p: p + self.w], p) for p in range(0, tam - self.w + 1)]:
            if chave not in res: res[chave] = []
            res[chave].append(offset)
        return res

    def get_all_offsets(self, s1, s2):
        w = len(s1)
        res = []
        for p in range(0, len(s2) - w + 1):
            if s2[p : p + w] == s1:
                res.append(p)
        return res

    def hits(self, query_map, sequences):
        res = []
        for chave, offsets in query_map.items():
            for o_query in offsets:
                for o_seq in self.get_all_offsets(chave, sequences):
                    res.append((o_query, o_seq))
        return res

    def extend_hit_dir(self, query, seq, o1, o2, direction):
        matches = 0
        count = 0
        while o1 >= 0 and o2 >= 0 and o1 < len(query) and o2 < len(seq):
            matches += 1 if query[o1] == seq[o2] else 0
            count += 1
            if 2 * matches < count:
                return o1, o2, matches, count - 1
            o1 += direction
            o2 += direction
        return o1 - direction, o2 - direction, matches, count

    def extend_hit(self, query, seq, hit, w):
        o1, o2 = hit
        left  = self.extend_hit_dir(query, seq, o1 - 1, o2 - 1, -1)
        right = self.extend_hit_dir(query, seq, o1 + w, o2 + w, +1)

        O1, O2, ML, SL = left
        _,   _, MR, SR = right

        return O1, O2, w + SL + SR, ML + w + MR

    def run_blast(self, query):
        if self.search_db(query):
            query_map = self.query_map(query)
            all_hits = self.hits(query_map, self.sequences)
            final_hits = []
            for hit in all_hits:
                extend_hit_res = self.extend_hit(query, self.sequences, hit, self.w)
                final_hits.append((query, hit[0], hit[1], extend_hit_res[2], extend_hit_res[3]))
            return final_hits
        else:
            return "No results found."


In [10]:
blast = Blast(15)
query = input("Enter a sequence: ")
results = blast.run_blast(query)
print(results)

Enter a sequence: TACCTT
[]


In [50]:
import mysql.connector as SQLC
class Blast:
    def __init__(self, w,sequence):
        self.w = w
        self.cnx = SQLC.connect(
            host ="geo.di.uminho.pt",
            user ="bioinformatica",
            password ="20221207",
            database ="AP_db_KRG"
        )
        self.sequence = sequence
    def query_map(self, query):
        tam = len(query)
        res = {}
        for chave, offset in [(query[p: p + self.w], p) for p in range(0, tam - self.w + 1)]:
            if chave not in res: res[chave] = []
            res[chave].append(offset)
        return res
        
    def get_all_offsets(self, s1, s2):
        w = len(s1)
        res = []
        for p in range(0, len(s2) - w + 1):
            if s2[p : p + w] == s1:
                res.append(p)
        return res
    
    def hits(self, query_map, sequence):
        res = []
        for chave, offsets in query_map.items():
            for o_query in offsets:
                for o_seq in self.get_all_offsets(chave, sequence):
                    res.append((o_query, o_seq))
        return res
    
    def extend_hit_dir(query, seq, o1, o2, direction):
        matches = 0
        count = 0
        while o1 >= 0 and o2 >= 0 and o1 < len(query) and o2 < len(seq):
            matches += 1 if query[o1] == seq[o2] else 0
            count += 1
            if 2 * matches < count:
                return o1, o2, matches, count - 1
            o1 += direction
            o2 += direction
        return o1 - direction, o2 - direction, matches, count
    
#     def extend_hit(query, seq, hit, w):
#         o1, o2 = hit
#         left  = extend_hit_dir(query, seq, o1 - 1, o2 - 1, -1)
#         right = extend_hit_dir(query, seq, o1 + w, o2 + w, +1)
#         O1, O2, ML, SL = left
#         _,   _, MR, SR = right

#         print({'ML':ML, 'SR':SR, 'SL':SL,'MR':MR})
#         return O1, O2, w + SL + SR, ML + w + MR


    def best_hit(self, query, sequence, w):
        query_map = self.query_map(query)
        hits = self.hits(query_map, sequence)
        best_score = -1
        best_hit = None
        for hit in hits:
            o1, o2, matches, count = self.extend_hit_dir(query, sequence, hit[0], hit[1], 1)
            o1_start, o2_start = o1 - count + 1, o2 - count + 1
            o1, o2, matches, count = self.extend_hit_dir(query, sequence, o1_start, o2_start, -1)
            score = matches / count
            if score > best_score or (score == best_score and count < best_hit[2]):
                best_score = score
                best_hit = (o1_start, o2_start, count, matches)
        return best_hit



#     def run_blast(self, query):
#         cursor = self.cnx.cursor()
#         cursor.execute("SELECT sequence FROM Gene")
#         sequences = cursor.fetchall()
#         cursor.close()
#         print(sequences)
#         if sequences:
#             qm = self.query_map(query)
#             all_offsets = []
#             all_hits = []
#             for sequence in sequences:
#                 all_offsets.extend(self.get_all_offsets(query,sequence))
#                 all_hits.extend(self.hits(qm, sequence))
                
#             return all_hits
#         else:
#             return "No results found."


In [53]:
class Blast:
    def __init__(self, w, seq):
        self.w = w
        self.seq = seq

    def query_map(self, query):
        query_map = {}
        for i in range(len(query) - self.w + 1):
            window = query[i: i + self.w]
            if window in query_map:
                query_map[window].append(i)
            else:
                query_map[window] = [i]
        return query_map

    def hits(self, query_map, sequence):
        hits = []
        for window, query_offsets in query_map.items():
            if window in sequence:
                seq_offset = sequence.find(window)
                while seq_offset != -1:
                    for query_offset in query_offsets:
                        hits.append((query_offset, seq_offset))
                    seq_offset = sequence.find(window, seq_offset + 1)
        return hits

    def extend_hit_dir(self, query, sequence, q_offset, s_offset, direction):
        count = 0
        matches = 0
        while q_offset >= 0 and s_offset >= 0 and q_offset < len(query) and s_offset < len(sequence) and query[q_offset] == sequence[s_offset]:
            count += 1
            matches += 1 if query[q_offset] == sequence[s_offset] else 0
            q_offset += direction
            s_offset += direction
        return q_offset - direction, s_offset - direction, matches, count

    def best_hit(self, query, sequence, w):
        query_map = self.query_map(query)
        hits = self.hits(query_map, sequence)
        best_score = -1
        best_hit = None
        for hit in hits:
            o1, o2, matches, count = self.extend_hit_dir(query, sequence, hit[0], hit[1], 1)
            o1_start, o2_start = o1 - count + 1, o2 - count + 1
            o1, o2, matches, count = self.extend_hit_dir(query, sequence, o1_start, o2_start, -1)
            score = matches / count
            if score > best_score or (score == best_score and count < best_hit[2]):
                best_score = score
                best_hit = (o1_start, o2_start, count, matches)
        return best_hit

In [55]:
query = "AATATAT"
seq = "AATATGTTATATAATAATATTT"
w = 3
blast = Blast(w, seq)
# result = blast.hits(blast.query_map(query), seq)
# print(result)
results = blast.best_hit(query, seq, w)
print(results)

(0, 0, 1, 1)


In [73]:
query = "TACCTTG"
w = 3
for i in sequences:
    seq=''.join(i)
    blast = Blast(w, seq)
    results = blast.best_hit(query, seq, w)
    print(results)

(0, 0, 1, 1)
None
(1, 11, 1, 1)
(2, 17, 1, 1)
None
(0, 10, 1, 1)
(2, 16, 1, 1)
(0, 23, 1, 1)
(0, 45, 1, 1)
None
None
(3, 29, 1, 1)
(0, 27, 1, 1)
(1, 35, 1, 1)
(2, 17, 1, 1)


In [5]:
def search_db(query):
    cnx = SQLC.connect(
            host ="geo.di.uminho.pt",
            user ="bioinformatica",
            password ="20221207",
            database ="AP_db_KRG")
    cursor = cnx.cursor()
    cursor.execute("SELECT sequence FROM Gene WHERE sequence LIKE %s", ('%'+query+'%',))
    sequences = cursor.fetchall()
    print(sequences)
    cursor.close()
    return sequences
search_db("AA")

[('TACCTTGTCCGAAAAGGAAAGACAAATCAAGAAGCAAACTGCTTTGGTTG',), ('CAACACGGGAAACCTCACCC',), ('GGGATGGAAGATGGCGGCCTA',), ('TCACATGGAGTACCACTAAGTTATTTGTAATATTTAATAAAGGAAATCTC',), ('GTTTGGTGAACTTCTGCCTA',), ('CCAAGAACAAATAACTTGTAAAATACACTGTTTTTTCACTGCAAATGTGT',), ('TGTGAATGAAAAAAGCAGCCAGTCAGTAGGAAACACTGTCTTGTATACAC',), ('TTTTTTTTTTTTTTTAACAATGGAGAAATCCATCAAAACATGGGTCATTT',), ('TTTTTTAAGTCAGTAAAAAGTAGGCACATCTTTATTTGGCAATGTCATAT',), ('GAATTCGTGCGCAACGCAATCCGGAAGTACAAGCAACTGCAGAAGCTGGT',), ('GGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATC',), ('NNNNNNNAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCCTA',)]


[('TACCTTGTCCGAAAAGGAAAGACAAATCAAGAAGCAAACTGCTTTGGTTG',),
 ('CAACACGGGAAACCTCACCC',),
 ('GGGATGGAAGATGGCGGCCTA',),
 ('TCACATGGAGTACCACTAAGTTATTTGTAATATTTAATAAAGGAAATCTC',),
 ('GTTTGGTGAACTTCTGCCTA',),
 ('CCAAGAACAAATAACTTGTAAAATACACTGTTTTTTCACTGCAAATGTGT',),
 ('TGTGAATGAAAAAAGCAGCCAGTCAGTAGGAAACACTGTCTTGTATACAC',),
 ('TTTTTTTTTTTTTTTAACAATGGAGAAATCCATCAAAACATGGGTCATTT',),
 ('TTTTTTAAGTCAGTAAAAAGTAGGCACATCTTTATTTGGCAATGTCATAT',),
 ('GAATTCGTGCGCAACGCAATCCGGAAGTACAAGCAACTGCAGAAGCTGGT',),
 ('GGAAGTAAAAGTCGTAACAAGGTCTCCGTTGGTGAACCAGCGGAGGGATC',),
 ('NNNNNNNAGAGTTTGATCCTGGCTCAGGACGAACGCTGGCGGCGTGCCTA',)]